In [ ]:
# import packages:
import numpy as np                   # Numeric Python
import matplotlib.pyplot as plt      # Plotting routines
import h5py                          # general HDF5 reading/writing library
import rioxarray as rx               # Package to read raster data from hdf5 files
from pyproj import Transformer, CRS  # libraries to allow coordinate transforms
import glob                          # Package to locate files on disk
import os                            # File-level utilities
import re                            # regular expressions for string interpretation
import icepyx as ipx                 # Package to interact with ICESat-2 online resources
from sliderule import icesat2        # Package for online ICESat-2 processing

In [ ]:
%matplotlib widget

In [ ]:
# logins, etc.

#HOST = 'https://urs.earthdata.nasa.gov'
#ipx.core.Earthdata.Earthdata('ben_smith','whatever@whatever.io', HOST).login()

url="icesat2sliderule.org"
icesat2.init(url, verbose=False)

In [ ]:
# Annika's bounding box:
# x,y
#-340,-80
#-480,-170

XR= np.array([-480, -340])*1.e3
YR= np.array([-170, -80])*1.e3

# shrink down to a tiny box in the center:
#XR=np.mean(XR)+np.array([-5.e3, 5.e3])
#YR=np.mean(YR)+np.array([-5.e3, 5.e3])


# Prepare coordinate transformations between lat/lon and polar stereographic
crs=CRS.from_epsg(3031)
to_xy_crs=Transformer.from_crs(crs.geodetic_crs, crs)
to_geo_crs=Transformer.from_crs(crs, crs.geodetic_crs)

corners_lat, corners_lon=to_geo_crs.transform(np.array(XR)[[0, 1, 1, 0, 0]], np.array(YR)[[0, 0, 1, 1, 0]])
latlims=[np.min(corners_lat), np.max(corners_lat)]
lonlims=[np.min(corners_lon), np.max(corners_lon)]

In [ ]:
# run a slideRule ATL06 query.  Just ask for cycle 8 (Antarctic winter, 2020)
# to avoid getting swamped right away

# See parameters here:
# http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html
params= { 'poly':[{'lon':this_lon, 'lat':this_lat} for this_lon, this_lat in zip(corners_lon, corners_lat)],
        'srt':3,
        'cnf':1,
        'len':10,
         'res':10,
         'ats':5,
         'cnt':10,
         'cycle':8,
         'maxi': 10,
        'pass_invalid':False}

D_IS_SR=icesat2.atl06p(params, 
                     asset="nsidc-s3")

In [ ]:
lon=np.array([gi.x for gi in D_IS_SR.geometry])
lat=np.array([gi.y for gi in D_IS_SR.geometry])
xy=to_xy_crs.transform(lat, lon)


In [ ]:
D_IS_SR

In [ ]:
# DO NOT RUN BIG SCATTER PLOTS WITH THE WHOLE AREA 
#plt.figure(); plt.scatter(xy[0], xy[1], c=D_IS_SR['h_mean'], cmap='Spectral'); plt.colorbar(label='surface height')
#plt.figure(); plt.scatter(xy[0], xy[1], c=D_IS_SR['rms_misfit'], vmin=0.1, vmax=1); plt.colorbar(label='rms_misfit')


In [ ]:

# pull the x, y, spot, and number of points from the data frame
sigma=np.array(D_IS_SR['rms_misfit'])
spot=np.array(D_IS_SR['spot'])
N=np.array(D_IS_SR['n_fit_photons'])
W=np.array(D_IS_SR['w_surface_window_final'])

# find the points that have 100 photons, from the strong spots, for which the surface window converged to 3 m
good=(N>100) & (np.in1d(spot, [1, 3, 5])) & (W==3.)
# make subsetted variables
xy1=[xy[0][good], xy[1][good]]
sigma1 = sigma[good]

# round the point locations to the nearest 5-km grid cell
xy_r = np.round((xy1[0]+1j*xy1[1])/5.e3)*5.e3
# find the unique values of the rounded grid locations
u_xy=np.unique(xy_r)

# make a list variable to hold the outputs
xy_rough=[]

# loop over the rounded locations:
for xi in u_xy:
    ii = xy_r==xi
    xy_rough += [(np.real(xi), np.imag(xi), np.median(sigma1[ii]))]

# Stick the tuples in the ouput list together to make a numpy array
xy_rough=np.c_[xy_rough]

In [ ]:
# map it!
North_box = dict(x = [-450, -375, -375, -450, -450], y = [-100, -115, -100, -90, -100], name='North')
South_box = dict(x = [ -450, -375, -375, -450, -450,], y = [-100, -115, -140, -115, -100], name='South')

ii=np.argsort(xy_rough[:,2])
plt.figure(); plt.scatter(xy_rough[:,0][ii], xy_rough[:,1][ii], c=xy_rough[:,2][ii], vmin=0.15, vmax=0.25, zorder=0); plt.colorbar()
for box in [North_box, South_box]:
    plt.plot(np.array(box['x'])*1000, np.array(box['y'])*1000, linewidth=2, zorder=10, label=box['name'])
                                                                                                 
plt.legend()
